In [5]:
### LLM(Large language model) 호출
from langchain.llms import OpenAI # text-davinci-003
from langchain.chat_models import ChatOpenAI # gpt-3.5-turbo

# llm = OpenAI()
# result = llm.predict("태양계 행성은 총 몇개가 있어?")

# temperature=> 얼마나 창의적 + 무작위적 으로 하는가에 대한 수치 높을 수록 더 창의적 이고 무작위 적으로 생성됨.
chat = ChatOpenAI(temperature=0.1)
result = chat.predict("태양계 행성은 총 몇개가 있어?")

result

'태양계 행성은 총 8개가 있습니다. 이는 수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 해왕성입니다.하지만 명왕성은 2006년에 왜곡체로 지정되어 행성에서 제외되었습니다.'

In [8]:
# Predict Messages
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="당신은 지리 전문가입니다. 그리고 당신은 한글으로만 답변합니다."),
    AIMessage(content="안녕하세요, 제 이름은 파올로입니다!"),
    HumanMessage(content="멕시코와 태국 사이의 거리는 어느정도 되나요? 그리고 당신의 이름은 무엇인가요?")
]
chat.predict_messages(messages)


AIMessage(content='멕시코와 태국 사이의 직선 거리는 약 15,000km 정도입니다. 제 이름은 파올로입니다.')

In [10]:
# Prompt Templates
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}")

prompt = template.format(country_a="Korea", country_b="Japan")
# chat.predict(prompt)

template = ChatPromptTemplate.from_messages([
    ("system",
     "You are a geography expert. And you only reply in {language}."),
    ("ai", "Hello, my name is {name}"),
    ("human",
     "What is the distamce between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Korea",
    name="mumusic",
    country_a="Japan",
    country_b="Korea",
)
chat.predict_messages(prompt)

AIMessage(content='일본과 한국 사이의 거리는 대략 900km입니다. 제 이름은 무뮤식입니다.')

# OutputParser and LCEL
### 사용하는 이유
- LLM의 응답을 변형해야 할 떄가 있기 떄문임.
- LLM은 항상 텍스트로만 응답하기에 응답을 변환 시켜주는 로직이 필요함.

In [35]:
# OutputParser and LCEL
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser  # base class 같은거임. 변형해서 사용 가능함.


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        # strip => 공백 제거
        items = text.split(",")
        return list(map(str.strip, items))


parser = CommaOutputParser()
parser.parse("a, b, c ")

template = ChatPromptTemplate.from_messages([
    ("system",
     "당신은 list 생성 기계 입니다. 입력받은 질문들은 모두 comma로 구분된 list로 답해주세요. 최대{max_items}만큼요. 전부다 소문자로 반환해주세요. list가 아닌 것으로는 답을 하지 마세요."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=3, question="디자인에 사용할만한 파스텔 색상을 알려주세요!")
# chat = ChatOpenAI(model="gpt-4" , temperature=0.1)
chat.predict_messages(prompt)

# -> 방향으로 return 값이 가서 마지막 값이 반환됨! chain!!
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":  5,
    "question": "디자인에 사용할만한 파스텔 색상을 알려주세요!"
})

['"baby pink"', '"mint green"', '"lavender"', '"peach"', '"sky blue"']

## Chaining Chains

### LangChain expression language (표현 언어)
- `template | chat | CommaOutputParser()` 이런 코드가 가능한 이유 ( 5 ~ 8 줄의 코드만으로 시간이 절약 되는 마법같은.. )
- `expression language` 는 구성요소(component)가 있음. -> chain이라 함.
- component는 input type이 정해져 있음. 이전 코드의 template 은 dictionary 로 정해져 있음.. good .. 사용한 순서로 dictionary , chat model(single string) , output parser(LLM, Chatmodel) 등! 사용한건 3가지 지만 5가지가 있음!
- `chain`은 `component`의 그룹 혹은 나열 이라고 보면 됨! `component`에서 실행된 값이 다음 `component`로 이어져서 넘어가고 그런식으로 동작함! 이때 `component`는 있는 순서에 따라 역활이 달라지는것!
- `chain`은 다른 `chain`과도 연결이 가능함.. (good..) 밑에 코드 예시 있음 !!

### etc (?)
- Langchain Python 은 내부에서 OpenAI Python package를 사용하고 있다!


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# 실시간으로 볼수 있게 streaming 을 true로 하고 callbacks로 화면에 출력!
chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients. response language is only korean"),
    ("human", "I want to cook {cuisine} food.")
])
# chain prompt -> chat
chef_chain = chef_prompt | chat

# chain 연결을 사용해 보기 위해 채식 주의자를 위한 chain 을 생성.
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

# 다른 chain 을 연결 ! 이때 chef_prompt 의 결과 값이 veg_chef_prompt 의 recipe 로 가게 해야함. => 오브젝트로 그냥 값을 전달해 주면 됨.. (이럴수가..)
final_chain = {"recipe": chef_chain} | veg_chain

# invoke는 실행! 근데 이때 chain에서 chain으로 값(recipe)이 위에서 설정해 놨기에 문제 없이 동작함! ( 이럴수가.. )
final_chain.invoke({
    "cuisine": "korean",
})

안녕하세요! 한국 음식을 요리하고 싶으시군요. 한국 음식 중에서 쉽고 맛있는 요리 하나 추천해 드릴게요. 바로 떡볶이입니다. 떡볶이는 한국인들이 가장 좋아하는 길거리 음식 중 하나로 유명합니다. 준비물과 요리법을 알려드리겠습니다.

재료:
- 떡 300g
- 양파 1개
- 어묵 100g
- 대파 1대
- 청양고추 2개
- 고추장 3큰술
- 고춧가루 1큰술
- 설탕 1큰술
- 국물용 멸치 5마리
- 물 4컵
- 김가루 (선택적)

요리법:
1. 양파, 어묵, 대파, 청양고추를 썰어 준비합니다.
2. 냄비에 물과 멸치를 넣고 끓여 멸치 육수를 만듭니다.
3. 끓는 물에 떡을 넣고 5분 정도 삶아줍니다.
4. 떡이 부드러워지면 물을 버리고 멸치 육수를 넣고 끓입니다.
5. 고추장, 고춧가루, 설탕을 넣고 잘 섞어줍니다.
6. 양파, 어묵, 대파, 청양고추를 넣고 끓입니다.
7. 떡볶이가 끓어오르면 김가루를 뿌려 마무리합니다.

이렇게 간단하게 맛있는 한국 떡볶이를 만들 수 있어요. 매운 맛을 원하시면 고춧가루를 더 넣어주세요. 즐거운 한국 음식 요리 되시길 바랍니다!한국 떡볶이는 정말 맛있는 음식이죠! 떡볶이를 즐기시되 채식주의자이신 경우, 몇 가지 대체재료를 사용하여 맛을 유지할 수 있어요. 

- 어묵: 어묵은 대체로 채식주의자들이 섭취 가능한 재료입니다. 대체재료로는 모두 굽은 두부나 마른 두부를 사용할 수 있어요. 두부를 썰어서 어묵 대신 사용하면 된답니다.
- 멸치: 멸치는 해조류나 버섯으로 대체할 수 있어요. 해조류나 버섯으로 육수를 만들어 떡볶이를 조리하면 멸치의 감칠맛을 대체할 수 있어요.
- 김가루: 김가루는 김을 볶아서 갈아 만든 가루인데, 대체재료로는 깨나 들깨, 혹은 볶은 참깨를 갈아서 사용할 수 있어요.

이렇게 대체재료를 사용하여 한국 떡볶이를 맛있게 즐기실 수 있어요. 즐거운 한국 음식 요리 되시길 바랍니다!

AIMessageChunk(content='한국 떡볶이는 정말 맛있는 음식이죠! 떡볶이를 즐기시되 채식주의자이신 경우, 몇 가지 대체재료를 사용하여 맛을 유지할 수 있어요. \n\n- 어묵: 어묵은 대체로 채식주의자들이 섭취 가능한 재료입니다. 대체재료로는 모두 굽은 두부나 마른 두부를 사용할 수 있어요. 두부를 썰어서 어묵 대신 사용하면 된답니다.\n- 멸치: 멸치는 해조류나 버섯으로 대체할 수 있어요. 해조류나 버섯으로 육수를 만들어 떡볶이를 조리하면 멸치의 감칠맛을 대체할 수 있어요.\n- 김가루: 김가루는 김을 볶아서 갈아 만든 가루인데, 대체재료로는 깨나 들깨, 혹은 볶은 참깨를 갈아서 사용할 수 있어요.\n\n이렇게 대체재료를 사용하여 한국 떡볶이를 맛있게 즐기실 수 있어요. 즐거운 한국 음식 요리 되시길 바랍니다!')

streming은 한글자 라도 볼 수 있게 되면, 그 값을 얻을 수 있음.
callback은 볼 수 잇는 문자가 생길 때 마다 print해줌.